# Univariate LSTM Models

## Data Preparation

In [1]:
import numpy as np

In [2]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        #check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [3]:
# Example
# 1. Define input sequence
raw_x = np.arange(10, 100, 10)
# 2. Choose a number of time steps
n_steps = 3
# 3. Split into samples
X, y = split_sequence(raw_x, n_steps)
# 4. Summarize the data
for i in range(len(X)):
    print(X[i], y[i])

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


## Vanilla LSTM

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [5]:
# parameters
n_steps = 3
n_features = 1

In [6]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [7]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [8]:
print("Shape of X matrix: ")
print(X.shape)

Shape of X matrix: 
(6, 3, 1)


In [9]:
# fit model
model.fit(X, y, epochs=200, verbose=0)

In [10]:
# Example (prediction)
x_input = np.array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)

In [11]:
print("Prediction (Vanilla LSTM):")
print(yhat[0,0])

Prediction (Vanilla LSTM):
103.190926


## Stacked LSTM

In [12]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [13]:
# fit model
model.fit(X, y, epochs=200, verbose=0)

In [14]:
# Example (prediction)
x_input = np.array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)

In [15]:
print("Prediction (Stacked LSTM):")
print(yhat[0,0])

Prediction (Stacked LSTM):
102.72347


## Bidirectional LSTM

In [16]:
from tensorflow.keras.layers import Bidirectional

In [17]:
# define model
model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [18]:
# fit model
model.fit(X, y, epochs=200, verbose=0)

In [19]:
# Example (prediction)
x_input = np.array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)

In [20]:
print("Prediction (Bidirectional LSTM):")
print(yhat[0,0])

Prediction (Bidirectional LSTM):
101.70916


## CNN LSTM 

In [100]:
from tensorflow.keras.layers import TimeDistributed, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D

In [70]:
print("Raw data: ")
print(raw_x)

Raw data: 
[10 20 30 40 50 60 70 80 90]


Required structure for input data:
```
[samples, subsequences, timesteps, features]
```

In [71]:
# choose a number of time steps
n_steps = 4
# split into samples
X, y = split_sequence(raw_x, n_steps)

In [72]:
print("X splitated samples: ")
print(X)
print(" \ny output sample: ")
print(y)

X splitated samples: 
[[10 20 30 40]
 [20 30 40 50]
 [30 40 50 60]
 [40 50 60 70]
 [50 60 70 80]]
 
y output sample: 
[50 60 70 80 90]


In [73]:
# reshape from [samples, timesteps] int [samples, subsequences, timesteps, features]
n_features = 1
n_seq = 2
n_substeps = 2
X = X.reshape((X.shape[0], n_seq, n_substeps, n_features))

**Important**
\begin{align}
n\_seq * n\_substeps = n\_steps
\end{align}

In [74]:
print("Reshaped X dimension: ")
print(X.shape)

Reshaped X dimension: 
(5, 2, 2, 1)


In [75]:
print("X: ")
print(X)

X: 
[[[[10]
   [20]]

  [[30]
   [40]]]


 [[[20]
   [30]]

  [[40]
   [50]]]


 [[[30]
   [40]]

  [[50]
   [60]]]


 [[[40]
   [50]]

  [[60]
   [70]]]


 [[[50]
   [60]]

  [[70]
   [80]]]]


In [103]:
# define model
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=32, kernel_size=1, activation='relu'), input_shape=(None, n_substeps, n_features)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [104]:
# fit model
model.fit(X, y, epochs=500, verbose=0)

In [ ]:
# demonstrate prediction
x_input = np.array([60, 70, 80, 90])
x_input = x_input.reshape((1, n_seq, n_substeps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

In [105]:
print("Prediction (CNN LSTM):")
print(yhat[0,0])

Prediction (CNN LSTM):
101.70916


## ConvLSTM

In [124]:
from tensorflow.keras.layers import ConvLSTM2D

Required structures:
```
[samples, timesteps, rows, columns, features]
```

In [118]:
# choose a number of time steps
n_steps = 4
# split into samples
X, y = split_sequence(raw_x, n_steps)

In [119]:
print("Splitted X: ")
print(X)

Splitted X: 
[[10 20 30 40]
 [20 30 40 50]
 [30 40 50 60]
 [40 50 60 70]
 [50 60 70 80]]


In [120]:
# reshape from [smaples, timesteps] into [samples, timesteps, rows, columns, features]
n_features = 1
n_rows = 1
n_seq = 2
n_substeps = 2
X = X.reshape((X.shape[0], n_seq, n_rows, n_substeps, n_features))

**Important**
\begin{align}
n\_seq * n\_substeps = n\_steps \\
n\_rows = n\_features
\end{align}

In [117]:
print("Reshaped X: ")
print(X)

Reshaped X: 
[[[[[10]
    [20]]]


  [[[30]
    [40]]]]



 [[[[20]
    [30]]]


  [[[40]
    [50]]]]



 [[[[30]
    [40]]]


  [[[50]
    [60]]]]



 [[[[40]
    [50]]]


  [[[60]
    [70]]]]



 [[[[50]
    [60]]]


  [[[70]
    [80]]]]]


```
kernel_size = (rows, columns)
```

In [125]:
# define model
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1, 2), activation='relu', input_shape=(n_seq, n_rows, n_substeps, n_features)))
model.add(Flatten())
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [126]:
# fit model
model.fit(X, y, epochs=500, verbose=0)

In [127]:
# demonstrate prediction
x_input = np.array([60, 70, 80, 90])
x_input = x_input.reshape((1, n_seq, n_rows, n_substeps, n_features))
yhat = model.predict(x_input, verbose=0)

In [128]:
print("Prediction (ConvLSTM): ")
print(yhat)

Prediction (ConvLSTM): 
[[103.70068]]


# Multivariate LSTM Models